In [9]:
import pandas as pd
import numpy as np

# 1. Cargar datos desde Excel
df = pd.read_excel('/Users/AlexLaugesen/repos/TFG1/venv/FINAL_DATA.xlsx')

In [4]:
# 2. Listas de posiciones ofensivas y defensivas
offense_positions = ['QB', 'RB', 'WR', 'TE']
defense_positions = ['CB', 'S', 'LB', 'DT', 'DE']

# 3. Definir funciones de cálculo de performance

def calc_performance_offense(row):
    pos = row['Pos']
    if pos == 'QB':
        C1 = max(0, min(40, (row.get('NFL.Cmp%', 0) - 30) * 1.5))
        if row.get('NFL.Pass_Att', 0) > 0:
            C2 = max(0, min(30, (row['NFL.Pass_TD'] / row['NFL.Pass_Att'] * 600)))
            C3 = max(-25, min(0, -(row['NFL.Pass_Int'] / row['NFL.Pass_Att'] * 500)))
        else:
            C2 = C3 = 0
        C4 = max(0, min(30, row.get('NFL.Pass_Y/A', 0) * 3))
        return max(0, C1 + C2 + C3 + C4)
    elif pos in ['RB', 'WR', 'TE']:
        games = row.get('NFL.Games', 1) or 1
        C1 = max(0, min(20, (row.get('NFL.Rush_Yds', 0) / games * 0.2)))
        C2 = max(0, min(20, (row.get('NFL.Rec_Yds', 0) / games * 0.2)))
        rush_att = row.get('NFL.Rush_Att', 0) or 1
        rec_att = row.get('NFL.Rec', 0) or 1
        C3 = max(0, min(15, (row.get('NFL.Rush_TD', 0) / rush_att) * 400))
        C4 = max(0, min(15, (row.get('NFL.Rec_TD', 0) / rec_att) * 400))
        C5 = max(0, min(15, row.get('NFL.Rush_Y/A', 0) * 2))
        C6 = max(0, min(15, row.get('NFL.Y/R', 0) * 2))
        return C1 + C2 + C3 + C4 + C5 + C6
    else:
        return np.nan


def calc_performance_defense(row):
    pos = row['Pos']
    if pos in ['CB', 'S']:
        D1 = max(0, min(40, row.get('NFL.Def_Int', 0) * 15))
        D2 = max(0, min(20, row.get('NFL.TFL', 0) * 8))
        D3 = max(0, min(10, row.get('NFL.Sk', 0) * 20))
        D4 = max(0, min(20, row.get('NFL.Comb', 0) * 0.36))
        D5 = max(0, min(10, row.get('NFL.Solo', 0) * 0.25))
        return D1 + D2 + D3 + D4 + D5
    elif pos in ['LB', 'DT', 'DE']:
        D1 = max(0, min(30, row.get('NFL.Sk', 0) * 10))
        D2 = max(0, min(30, row.get('NFL.TFL', 0) * 6.5))
        D3 = max(0, min(20, row.get('NFL.Comb', 0) * 0.4))
        D4 = max(0, min(10, row.get('NFL.Solo', 0) * 0.5))
        D5 = max(0, min(10, row.get('NFL.Def_Int', 0) * 5))
        return D1 + D2 + D3 + D4 + D5
    else:
        return np.nan


def calc_performance(row):
    pos = row['Pos']
    if pos in offense_positions:
        return calc_performance_offense(row)
    elif pos in defense_positions:
        return calc_performance_defense(row)
    else:
        return np.nan
    
# 4. Calcular columna 'Performance' antes del one-hot encoding
df['Performance'] = df.apply(calc_performance, axis=1)

print("Distribución de la nueva variable 'Performance':")
display(df['Performance'].describe())

Distribución de la nueva variable 'Performance':


count    3246.000000
mean       32.386725
std        26.765856
min         0.000000
25%         8.000000
50%        29.539583
75%        51.591560
max       100.000000
Name: Performance, dtype: float64

In [8]:
# 5. One-Hot Encoding de variables categóricas
categorical_cols = ['Pos', 'Draft Team', 'Draft College']
df_encoded = pd.get_dummies(df, columns=categorical_cols, prefix=['Pos', 'DraftTeam', 'DraftCollege'])

# 6. Guardar la base procesada en CSV
df_encoded.to_csv('/Users/AlexLaugesen/repos/TFG1/venv/FINAL_DATA_processed.csv', index=False)

# 7. Segmentación en ofensivos y defensivos
# Usar any() para combinar condiciones sobre columnas one-hot
offense_cols = [f"Pos_{p}" for p in offense_positions]
defense_cols = [f"Pos_{p}" for p in defense_positions]

df_offense = df_encoded[df_encoded[offense_cols].any(axis=1)]
df_defense = df_encoded[df_encoded[defense_cols].any(axis=1)]

# Guardar segmentos
df_offense.to_csv('/Users/AlexLaugesen/repos/TFG1/venv/FINAL_DATA_offense.csv', index=False)
df_defense.to_csv('/Users/AlexLaugesen/repos/TFG1/venv/FINAL_DATA_defense.csv', index=False)

print("Procesamiento completado. Archivos guardados")


Procesamiento completado. Archivos guardados
